This scrapper is used to establish a .json file with the information of interest among the energy consuption invoices<br>
<a href='https://github.com/jcushman/pdfquery'>PDFquery</a><br>
<a href='http://okfnlabs.org/blog/2016/04/19/pdf-tools-extract-text-and-data-from-pdfs.html'>List of pdf scrappers</a>

In [3]:
import pdfquery
import re
import os
import json
import time
from datetime import datetime

In [62]:
log = list() #list of scrapped data
for filename in os.listdir(os.getcwd()+'\\faturas'):
    if filename.endswith(".pdf"): 
        pdf = pdfquery.PDFQuery('./faturas/'+filename)
        pdf.load()
        #pdf.tree.write("test.xml", pretty_print=True, encoding="utf-8") #for debbuging purposes
        
        #define a threshold between consumption and additional charge fields
        th_add = pdf.pq(':contains("Lançamentos e Serviços")')[2].layout.bbox[-1]
        
        #get sentences based on position in .pdf's
        data = pdf.extract([
                ('with_formatter', 'text'),
                ('nome', 'LTTextLineHorizontal:in_bbox("28.35, 756.943, 124.292, 770.033")'),
                ('conta','LTTextLineHorizontal:in_bbox("422.21, 653.888, 484.72, 677.328")'),
                ('valor','LTTextLineHorizontal:in_bbox("513.08,'+str(th_add)+', 541.422, 677.328")'),
                ('tarifa_add','LTTextLineHorizontal:overlaps_bbox("529.74, 611.368, 541.416, '+str(th_add)+'")'),
                ('distribuicao','LTTextLineHorizontal:in_bbox("296.37, 347.888, 311.382, 354.824")'),
                ('enc.setoriais','LTTextLineHorizontal:overlaps_bbox("343.14, 347.888, 358.152, 354.824")'),
                ('energia','LTTextLineHorizontal:overlaps_bbox("385.41, 347.888, 403.758, 354.824")'),
                ('trasmissao','LTTextLineHorizontal:overlaps_bbox("429.6, 347.888, 444.612, 354.824")'),
                ('tributos','LTTextLineHorizontal:in_bbox("472.12, 347.888, 487.132, 354.824")'),
                ('total','LTTextLineHorizontal:overlaps_bbox("449.53, 696.003, 500.184, 709.093")'),
                ('un_consumidora', 'LTTextLineHorizontal:in_bbox("299.06, 80.781, 326.304, 88.873")'),
                ('data', 'LTTextLineHorizontal:in_bbox("386.93, 80.781, 412.228, 88.873")')
            ])
        
        #use regex to extract data
        schema = {
            'nome': data['nome'],
            'unidade': int(data['un_consumidora']),
            'data':  time.mktime(datetime.strptime(data['data'],'%m/%Y').timetuple()),
            'total': float(re.findall(r'-?[0-9]+\.[0-9]*', data['total'].replace(',','.'))[0]),
            'tarifa': list(map(lambda x: float(x), re.findall(r'[0-9]+(\.[0-9]*)', data['conta'].replace(',','.')))),
            'consumo': list(map(lambda x: int(x), re.findall(r'(?:^|(?<=\s))[0-9]+(?=$|\s)', data['conta']))),
            'valor': list(map(lambda x: float(x), re.findall(r'-?[0-9]+\.[0-9]*', data['valor'].replace(',','.'))))[:-1],
            'adicional': list(map(lambda x: float(x), re.findall(r'-?[0-9]+\.[0-9]*', data['tarifa_add'].replace(',','.'))))[:-1],
            'comp_distribuicao': float(data['distribuicao'].replace(',','.')),
            'comp_enc.setoriais': float(data['enc.setoriais'].replace(',','.')),
            'comp_energia': float(data['energia'].replace(',','.')),
            'comp_transmissao': float(data['trasmissao'].replace(',','.')),
            'comp_tributos': float(data['tributos'].replace(',','.'))
        }
       
        log.append(schema)
    else:
        continue

#dump log as a .json file
with open('database.json', 'w') as file:
    json.dump({'logs': log}, file)